Introduction
Welcome to the Connecting Python to SQL lab!

In this lab, you will be working with the Sakila database on movie rentals. Specifically, you will be practicing how to do basic SQL queries using Python. By connecting Python to SQL, you can leverage the power of both languages to efficiently manipulate and analyze large datasets. Throughout this lab, you will practice how to use Python to retrieve and manipulate data stored in the Sakila database using SQL queries. Let's get started!

Challenge
In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

1. Establish a connection between Python and the Sakila database.

2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

    - engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
    - month: an integer representing the month for which rental data is to be retrieved.
    - year: an integer representing the year for which rental data is to be retrieved.

    The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

    The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

    Hint: Consider making use of pandas groupby()

4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [1]:
import pandas as pd
import mysql.connector
from getpass import getpass
from sqlalchemy import create_engine

password = getpass()


In [2]:
# 1. Establish a connection between Python and the Sakila database.
conn = mysql.connector.connect( host = "localhost",
                               user = "root",
                               password = password)

In [3]:
conn.is_connected() # The connection was successful

True

In [4]:
# Creation of the cursor to retrieve queries
cursor = conn.cursor()

In [5]:
#2. 
def rentals_count(month, year):
    query = f"""SELECT *
                FROM sakila.rental
                WHERE YEAR(rental_date)={year} AND MONTH(rental_date)={month}"""
    cursor.execute(query)
    results = cursor.fetchall()
    return pd.DataFrame(results, columns = cursor.column_names)

In [69]:
rentals_count(5,2005)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [70]:
# 3. 
def rental_count_month(df,month, year):
    new_column=df.groupby("customer_id").agg({"rental_id":"count"})
    df_2=pd.merge(df,new_column["rental_id"],on = "customer_id", how = "left")
    df_2.rename(columns = {"rental_id_x": "rental_id",
                     "rental_id_y": f"rentals_0{month}_{year}"},inplace = True)
    
    return df_2
    

In [179]:
df_a=rental_count_month(rentals_count(5,2005),5,2005)
df_a

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,rentals_05_2005
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,2
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,1
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,3
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,1
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,5
...,...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53,7
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53,3
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53,6
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53,2


In [180]:
df_b=rental_count_month(rentals_count(6,2005),6,2005)
df_b

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,rentals_06_2005
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53,5
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53,6
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53,5
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53,3
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53,6
...,...,...,...,...,...,...,...,...
2306,3465,2005-06-21 22:10:01,1488,510,2005-06-30 21:35:01,1,2006-02-15 21:30:53,8
2307,3466,2005-06-21 22:13:33,371,226,2005-06-25 21:01:33,2,2006-02-15 21:30:53,2
2308,3467,2005-06-21 22:19:25,729,543,2005-06-27 00:03:25,2,2006-02-15 21:30:53,5
2309,3468,2005-06-21 22:43:45,2899,100,2005-06-30 01:49:45,1,2006-02-15 21:30:53,4


In [181]:
# 4.
def compare_rentals(df_1,df_2):
    df_1.drop(columns = ["rental_id","rental_date","inventory_id","return_date","staff_id","last_update"], inplace = True)
    df_1=df_1.drop_duplicates(subset = ["customer_id"], keep = "first")
    
    df_2.drop(columns = ["rental_id","rental_date","inventory_id","return_date","staff_id","last_update"], inplace = True)
    df_2=df_2.drop_duplicates(subset = ["customer_id"], keep = "first")

    df_combined=pd.merge(df_1,df_2, on="customer_id",how="outer")

    temp_list=[]
    for i in range(len(df_combined)-1):
        temp_value=abs(df_combined.loc[i][df_combined.columns[1]]-df_combined.loc[i][df_combined.columns[2]])
        temp_list.append(temp_value)


    temp_series = pd.Series(temp_list, name = "difference")

    df_combined["difference"]=temp_series


    return df_combined

In [183]:
compare_rentals(rental_count_month(rentals_count(5,2005),5,2005),rental_count_month(rentals_count(6,2005),6,2005))
# compare_rentals(df_a,df_b)

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,130,2.0,6.0,4.0
1,459,1.0,7.0,6.0
2,408,3.0,3.0,0.0
3,333,1.0,4.0,3.0
4,222,5.0,2.0,3.0
...,...,...,...,...
593,598,NaN,1.0,NaN
594,487,NaN,1.0,NaN
595,555,NaN,1.0,NaN
596,335,NaN,1.0,NaN
